In [1]:
import torch
from torch import tensor


Whereas before we vectorized away the innermost loop of matmul, in this version we first focus on parallelizing the outer loops that iterae through the entries of the target array. Each invocation accepts a `grid = (i, j)` parameter to indicate the target coordinate.

In [2]:
def matmul(grid, a, b, c):
    i, j = grid
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]):
            tmp += a[i, k] * b[k, j]
        c[i,j] = tmp


In [3]:
from fastai.vision.all import *

pickle_path = URLs.path('mnist_png')/'mnist_png.pkl'
path = untar_data(URLs.MNIST)/'training'

if not pickle_path.exists():
    pickle_path.parent.mkdir(parents=True, exist_ok=True)
    ds = DataBlock(
        blocks = (ImageBlock(PILImageBW), CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        splitter = RandomSplitter(1/6, seed=0)
    ).datasets(path)

    xs, ys = zip(*ds.train, *ds.valid)
    xs = np.stack(L(map(lambda x: np.array(x, dtype=np.float32).reshape(-1), xs))) / 255.
    ys = np.array(ys, dtype=np.int64)

    x_train, x_valid = xs[:len(ds.train)], xs[len(ds.train):]
    y_train, y_valid = ys[:len(ds.train)], ys[len(ds.train):]

    save_pickle(pickle_path, [x_train, y_train, x_valid, y_valid])

    del ds, xs, ys, x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = map(tensor, load_pickle(pickle_path))

torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)

m1 = x_valid[:5]
m2 = weights
ar, ac = m1.shape 
br, bc = m2.shape

t1 = torch.zeros(ar, bc)

for i in range(ar):         # 5
    for j in range(bc):     # 10
        for k in range(ac): # 784
            t1[i, j] += m1[i, k] * m2[k, j]


In [4]:
res = torch.zeros(ar, bc)
matmul((0, 0), m1, m2, res)
res


tensor([[-7.8318,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000]])

We define a harness for calling `matmul` on each entry.

In [5]:
def launch_kernel(kernel, grid_x, grid_y, *args, **kwargs):
    for i in range(grid_x):
        for j in range(grid_y):
            kernel((i, j), *args, **kwargs)


We give it a go.

In [6]:
res = torch.zeros(ar, bc)
launch_kernel(matmul, ar, bc, m1, m2, res)
res


tensor([[ -7.8318,  -9.3646,   6.0571, -14.2204,  20.7075,   1.9792, -20.8511,
         -14.9247, -24.9726,  -1.5234],
        [  9.5953,   8.4150,   8.4275,  -7.7321,  11.0846, -26.4744,  -3.6933,
          -9.4670, -35.0040,   7.3345],
        [  4.7560,  -3.3547,  -8.8216,  11.5083,  -5.3496,  -2.2799, -14.0786,
           1.3523,  -7.9863,  -3.3055],
        [  6.8275,   1.2459,  -0.5831,  13.8737,  -1.6997,  -7.8282,  -1.4413,
           0.4070, -17.4009,  -7.7779],
        [-11.6686, -12.9786,  -7.4251,  -4.4291,  -1.6932,   3.1152,  -5.8956,
          -5.2578, -15.3839,   3.5509]])

In [7]:
import numpy as np
from numba import cuda
from fastcore.test import *

Using `cuda.jit`, the indices are provided by `cuda.grid(2)`. In general, the GPU may attempt to evaluate the function beyond the shape of `c`, so we 

In [8]:
@cuda.jit
def matmul(a,b,c):
    i, j = cuda.grid(2)
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp


We setup the testing environment, and send all the relevant tensors to the GPU.

In [9]:
tr = x_train @ weights
r = np.zeros(tr.shape)
m1g, m2g, rg = map(cuda.to_device, (x_train, weights, r))


We shall execute 16 x 16 threads for each CUDA block, and compute the number of blocks in each direction that we shall use. If `c` has shape strictly smaller than `blocks * TPB`, then `matmul` will be run on invalid indices, hence the bounds checking in our definition previously.

In [10]:
import math

TPB = 16
rr, rc = r.shape
blocks = (math.ceil(rr / TPB), math.ceil(rc / TPB))
blocks


(3125, 1)

Testing correctness.

In [11]:
matmul[blocks, (TPB, TPB)](m1g, m2g, rg)
r = rg.copy_to_host()
test_close(tr, r, eps=1e-4)


Testing performance.

In [12]:
%%timeit -n 10
matmul[blocks, (TPB, TPB)](m1g, m2g, rg)
r = rg.copy_to_host()


3.68 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Finally, we evaluate the speed of the inbuilt `matmul` of PyTorch on GPU.

In [13]:
m1c, m2c = x_train.cuda(), weights.cuda()


In [14]:
r = (m1c @ m2c).cpu()


Testing performance; we observe a close to 10x speedup.

In [15]:
%timeit -n 10 r = (m1c @ m2c).cpu()


576 µs ± 158 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
